In [2]:
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from PIL import Image
import io
import time
import numpy as np
import cv2
import os
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [3]:
#기업명
company_name = []
#기업설명
company_main=[]
#기업 매출
company_sale=[]
#기업 누적판매량
company_csale=[]
#기업 고용인원
company_employee=[]
#기업 카테고리
company_category=[]

1.스타트업 목록 크롤링

In [4]:
#스타트업 목록 크롤링
browser = webdriver.Chrome()
url = 'https://www.innoforest.co.kr/dataroom?orderByField=invstCd&page=1'
browser.get(url)
company_sale_people=[]
company_category_list=[]
page = 200
for p in range(1, page + 1):  # 원하는 페이지까지 반복문
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight/3);")
    time.sleep(0.75)
    page_source = browser.page_source
    bs = BeautifulSoup(page_source, 'html.parser')
    #전체 페이지 가져오기
    allDiv=bs.find('div',class_='css-5kzijo')
    #회사별로 가져오기
    company_body = allDiv.find_all('tr',class_='css-0')
    #회사 이름 가져오기
    for i in company_body:
        name_tags = i.find_all('p')
        if name_tags:
            for name_tag in name_tags:
                company_name.append(name_tag.get_text())
    #회사          
    for j in company_body:
        contents_tags=j.find_all('div',class_='css-pzsc88')
        company_a_tags=j.find_all('a')
      #회사 내용   
        for i in company_a_tags:
            company_main_tags=i.find('div',class_='css-pzsc88')
            if company_main_tags:
                company_main.append(company_main_tags.get_text())
        #매출,누적매출,고용인원 
        sale_tags=j.find_all('td',class_='css-1ytuqjc')
        for k in sale_tags:
            #누적매출
            company_sale_list=k.find('div',class_='css-pzsc88')
            if company_sale_list:
                company_csale.append(company_sale_list.get_text())
            #누적매출,고용인원
            else:
                company_sale_people.append(k.get_text())
        category_tags=j.find_all('td',class_='css-1wx7bxj')
        for m in category_tags:
            company_category_list.append(m.get_text())
    # for문 안에 page_bar를 넣어주어 매번 지정
    page_bar = browser.find_elements(By.CSS_SELECTOR,
                                      '#__next > main > div > div:nth-child(2) > div > article.css-zyaupy > div.css-owhe8m > div > span')
    next_page = browser.find_elements(By.CSS_SELECTOR,
                                      '#__next > main > div.css-c0cixj > div:nth-child(2) > div > article.css-zyaupy > div.css-owhe8m > div.css-vlib4i > a')
    for button in page_bar:
        if button.text == str(p):
            button.click()
            break
    
    if p % 5 == 0:  # 5의 배수일때 다음 5페이지 버튼 클릭
        if p == 5:
            time.sleep(0.5)
            next_page[0].click()
            time.sleep(0.75)
        else:
            next_page[1].click()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.93)
Stacktrace:
	GetHandleVerifier [0x00007FF7B0E61502+60802]
	(No symbol) [0x00007FF7B0DDAC02]
	(No symbol) [0x00007FF7B0C97CE4]
	(No symbol) [0x00007FF7B0C6DFDF]
	(No symbol) [0x00007FF7B0D11E57]
	(No symbol) [0x00007FF7B0D298D1]
	(No symbol) [0x00007FF7B0D0A923]
	(No symbol) [0x00007FF7B0CD8FEC]
	(No symbol) [0x00007FF7B0CD9C21]
	GetHandleVerifier [0x00007FF7B116411D+3217821]
	GetHandleVerifier [0x00007FF7B11A60B7+3488055]
	GetHandleVerifier [0x00007FF7B119F03F+3459263]
	GetHandleVerifier [0x00007FF7B0F1B846+823494]
	(No symbol) [0x00007FF7B0DE5F9F]
	(No symbol) [0x00007FF7B0DE0EC4]
	(No symbol) [0x00007FF7B0DE1052]
	(No symbol) [0x00007FF7B0DD18A4]
	BaseThreadInitThunk [0x00007FFAC57C7344+20]
	RtlUserThreadStart [0x00007FFAC59026B1+33]


In [ ]:
#매출과 고용인원 분리
for idx, value in enumerate(company_category_list):
    if idx % 2 == 1:
        company_category.append(value)

In [ ]:
#매출과 고용인원 분리
for idx, value in enumerate(company_sale_people):
    if idx % 2 == 0:
        company_sale.append(value)
    else:
        company_employee.append(value)

In [ ]:
company_category

In [ ]:
company = pd.DataFrame(data=[], columns=['기업이름','기업설명','누적투자금액','매출','고용인원','카테고리'])

for i in range(len(company_name)):
    tmp = []
    tmp.append(company_name[i])
    tmp.append(company_main[i])
    tmp.append(company_csale[i])
    tmp.append(company_sale[i])
    tmp.append(company_employee[i])
    tmp.append(company_category[i])
    tmp = pd.DataFrame(data=[tmp], columns = company.columns)
    company = pd.concat([company,tmp])
    
company.reset_index(inplace=True, drop=True)
company

In [ ]:
company.to_csv('company.csv', index=False, encoding='utf-8-sig')

In [79]:
company_capital=[]
company_invest_ct=[]
company_tech_rank=[]

2.스타트업 세부목록 크롤링

In [5]:
browser = webdriver.Chrome()
url = 'https://www.innoforest.co.kr/dataroom?orderByField=invstCd&page=1'
browser.get(url)
page = 6
for p in range(1, page + 1):  # 원하는 페이지까지 반복문
    page_source = browser.page_source
    bs = BeautifulSoup(page_source, 'html.parser')
    for k in range(1,4):
        css_selector = f'#__next > main > div.css-c0cixj > div:nth-child(2) > div > article.css-zyaupy > table > tbody > tr:nth-child({k}) > td.css-1hbg8y1 > a'
        company_link = browser.find_elements(By.CSS_SELECTOR, css_selector)
        if company_link:
            company_link[0].click()
            page_source = browser.page_source
            bs = BeautifulSoup(page_source, 'html.parser')
            time.sleep(2)
            #링크들어가서 크롤링 할 내용
            ###########################################
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")       
    
            browser.back()

    page_bar = browser.find_elements(By.CSS_SELECTOR,
                                      '#__next > main > div > div:nth-child(2) > div > article.css-zyaupy > div.css-owhe8m > div > span')
    next_page = browser.find_elements(By.CSS_SELECTOR,
                                      '#__next > main > div.css-c0cixj > div:nth-child(2) > div > article.css-zyaupy > div.css-owhe8m > div.css-vlib4i > a')
    
    for button in page_bar:
        if button.text == str(p):
            button.click()
            time.sleep(2)  # 클릭 후 2초 대기
            break
    
    if p % 5 == 0:  # 5의 배수일 때 다음 5페이지 버튼 클릭
        if p == 5:
            next_page[0].click()
        else:
            next_page[1].click()
        time.sleep(0.75)  # 페이지 이동 후 대기

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.93)
Stacktrace:
	GetHandleVerifier [0x00007FF7B0E61502+60802]
	(No symbol) [0x00007FF7B0DDAC02]
	(No symbol) [0x00007FF7B0C97CE4]
	(No symbol) [0x00007FF7B0C6DFDF]
	(No symbol) [0x00007FF7B0D11E57]
	(No symbol) [0x00007FF7B0D298D1]
	(No symbol) [0x00007FF7B0D0A923]
	(No symbol) [0x00007FF7B0CD8FEC]
	(No symbol) [0x00007FF7B0CD9C21]
	GetHandleVerifier [0x00007FF7B116411D+3217821]
	GetHandleVerifier [0x00007FF7B11A60B7+3488055]
	GetHandleVerifier [0x00007FF7B119F03F+3459263]
	GetHandleVerifier [0x00007FF7B0F1B846+823494]
	(No symbol) [0x00007FF7B0DE5F9F]
	(No symbol) [0x00007FF7B0DE0EC4]
	(No symbol) [0x00007FF7B0DE1052]
	(No symbol) [0x00007FF7B0DD18A4]
	BaseThreadInitThunk [0x00007FFAC57C7344+20]
	RtlUserThreadStart [0x00007FFAC59026B1+33]


In [88]:
browser = webdriver.Chrome()
url = 'https://www.innoforest.co.kr/dataroom?orderByField=invstCd&page=1'
browser.get(url)
page_source = browser.page_source
bs = BeautifulSoup(page_source, 'html.parser')
for k in range(1,4):
        css_selector = f'#__next > main > div.css-c0cixj > div:nth-child(2) > div > article.css-zyaupy > table > tbody > tr:nth-child({k}) > td.css-1hbg8y1 > a'
        company_link = browser.find_elements(By.CSS_SELECTOR, css_selector)
        if company_link:
            company_link[0].click()
            ##### url2= <클릭했을때 바뀐 링크 집어넣어야함>############
            browser.get(url2)
            page_source = browser.page_source
            bs = BeautifulSoup(page_source, 'html.parser')
        time.sleep(2)
        allDiv= bs.find('div',class_='css-1lf10gr')
        box=allDiv.find_all('dl',class_='css-17jidve')
        time.sleep(2)
            #링크들어가서 크롤링 할 내용
            ###########################################
        capitals=[]
        for i in box:
            capitals_tags=i.find_all('dd',class_='css-3qsen1')
            if capitals_tags:
                for capital_tag in capitals_tags:
                    capitals.append(capital_tag.get_text())
                company_capital.append(capitals[0][0] )#자본금
                company_invest_ct.append([capitals[3]][0])#투자유치횟수
                company_tech_rank.append([capitals[5]][0])#기술평균등급             
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")       
        browser.back()

AttributeError: 'NoneType' object has no attribute 'find_all'

In [80]:
capitals=[]
for i in box:
    capitals_tags=i.find_all('dd',class_='css-3qsen1')
    if capitals_tags:
        for capital_tag in capitals_tags:
            capitals.append(capital_tag.get_text())
        company_capital.append(capitals[0][0])
        company_invest_ct.append([capitals[3]][0])
        company_tech_rank.append([capitals[5]][0])

In [74]:
capitals_tags

[<dd class="css-3qsen1">55.4억원</dd>,
 <dd class="css-3qsen1">2,201명</dd>,
 <dd class="css-3qsen1">5,063억원</dd>,
 <dd class="css-3qsen1">7</dd>,
 <dd class="css-3qsen1">2.9조원</dd>,
 <dd class="css-3qsen1">2.8</dd>]

['2.8']